In [1]:
import numpy as np
from numpy.linalg import pinv
from enum import Enum

### Define Matrix Inversion Methods

In [2]:
class InverseMethods(Enum):
    PINV = 1
    DLS = 2
    SRINV = 3

In [3]:
def inverseSolver(A, method=InverseMethods.PINV, tolerance=1e-4, betaMax=1e-1):
    if method == InverseMethods.PINV:
        # method 1: matlab's pinv with a threshold
        A_inv = np.linalg.pinv(A, rcond=1e-8)

    elif method == InverseMethods.DLS:
        # method 2: damped least-square inverse
        lambda_val = tolerance

        A_inv = np.matmul(np.transpose(A), np.linalg.inv(np.matmul(A, np.transpose(A)) + lambda_val**2*np.eye(A.shape[0])))

    elif method == InverseMethods.SRINV:
        # method 3: singularity-robust inverse base on numerical filtering

        # initialize some constants
        beta = 0
        lambda_val = 0

        # get the minimum singular value from SVD
        U, S, V = np.linalg.svd(A)
        s_min = np.min(S)

        # compute the sume of the output vectors that correspond to
        # singular values below the threshold (tolerance)
        u_sum = np.zeros((A.shape[0], A.shape[0]))
        if s_min < tolerance:
            for ind in range(S.size - 1, -1, -1):
                if s_data[ind] < tolerance:
                    u_sum += np.outer(U[:, ind], U[:, ind])
                else:
                    break
            # scale beta coefficient with the minimum singular value
            # beta maps to the range of [0, betaMax] depending on the size of
            # minimum singular value with respect to the threshold.
            beta = (1-(s_min/tolerance)**2)*betaMax

        # compute the inverse with beta coefficient
        A_inv = np.matmul(np.transpose(A), np.linalg.inv(np.matmul(A, np.transpose(A)) + lambda_val**2*np.eye(A.shape[0]) + beta*u_sum))

    return A_inv

### Test the Inversion Methods

In [4]:
# Generate a random 2x3 matrix with values between 0 and 1
matrix = np.random.rand(2, 3)
print(matrix)

[[0.52901031 0.39703874 0.05249883]
 [0.28703472 0.23853752 0.66238773]]


In [5]:
# get the inverses from each method and compare the outputs
inv1 = inverseSolver(matrix, InverseMethods.PINV)
inv2 = inverseSolver(matrix, InverseMethods.DLS)
inv3 = inverseSolver(matrix, InverseMethods.SRINV)
print(inv1)
print(inv2)
print(inv3)

(2, 2)
[[ 1.28347035 -0.12808802]
 [ 0.92620656 -0.03811095]
 [-0.8897139   1.57891905]]
[[ 1.28347031 -0.12808799]
 [ 0.92620653 -0.03811093]
 [-0.88971385  1.578919  ]]
[[ 1.28347035 -0.12808802]
 [ 0.92620656 -0.03811095]
 [-0.8897139   1.57891905]]


### Test helper methods of eSNS IK Solver

In [126]:
# Find scale factor for given inputs
def findScaleFactor(low, upp, a):
    sMax = 1 # the maximum feasible scale factor
    sMin = 0 # the minimum feasible scale factor
    if abs(a) < 1e10 and abs(a) > 1e-10:
        if a < 0 and low < 0 and a <= upp:
            sMax = min(1, low/a)
            sMin = max(0, upp/a)
        elif a >= 0 and upp > 0 and a >= low:
            sMax = min(1, upp/abs(a))
            sMin = max(0, low/abs(a))
    return sMax, sMin

In [7]:
s = findScaleFactor(-0.1, 0.2, 0.3)
print(s)

(0.6666666666666667, 0)


In [8]:
A = np.random.rand(3, 3)
B = np.random.rand(3, 3)
C1 = np.matmul(A, B)
C2 = A.dot(B)
print(C1)
print(C2)

[[0.9954694  0.97980195 0.87049253]
 [0.48472076 1.17917854 0.55622594]
 [0.48592185 0.45555469 0.42830709]]
[[0.9954694  0.97980195 0.87049253]
 [0.48472076 1.17917854 0.55622594]
 [0.48592185 0.45555469 0.42830709]]


### Multi-task extended-SNS IK solver

In [110]:
def snsIk_vel_mt_extended(C, limLow, limUpp, dxGoalData, JData, invSolver=None):
    """
    Calculates joint velocities given a set of Jacobians and task goals
    using the inverse kinematics algorithm with velocity-based multi-task
    prioritization and constraints. This function uses a PINV solver to
    compute the pseudoinverse of the Jacobian matrix.

    Args:
    C (numpy.ndarray): Joint limits constraint matrix.
    limLow (numpy.ndarray): Joint lower limits.
    limUpp (numpy.ndarray): Joint upper limits.
    dxGoalData (list): List of numpy arrays, each containing the desired
        velocity for each task.
    JData (list): List of numpy arrays, each containing the Jacobian matrix
        for each task.
    invSolver (callable, optional): A function that computes the inverse of
        a matrix. Defaults to None, which sets the inverse solver to use the
        PINV method.

    Returns:
    tuple: A tuple containing the following elements:
        dq (numpy.ndarray): The joint velocities that achieve the task goals.
        sData (numpy.ndarray): An array containing the task scaling factors
            used to achieve the task goals.
        exitCode (int): A flag indicating whether the algorithm succeeded
            (1) or failed (0).
    """

    # use PINV method if the invMethod not specified
    if invSolver is None:
        invSolver = lambda A: inverseSolver(A, InverseMethods.PINV)

    # get the number of tasks
    nTask = len(JData)
    sData = np.zeros(nTask)

    # initialization
    exitCode = 1
    tol = 1e-6 # a tolerance used for various numerical checks
    tolTight = 1e-10 # a tighter tolerance
    nIterationsMax = 20

    nJnt = JData[0].shape[1]
    k = C.shape[0] - nJnt
    I = np.eye(nJnt)
    Pi = I
    Sact = np.zeros((nJnt + k, nJnt + k))
    dqi = np.zeros((nJnt, 1))
    w = np.zeros((nJnt + k, 1))
    Cact = Sact.dot(C)

    for iTask in range(nTask):

        # get i-th task jacobian
        Ji = JData[iTask]
        ndxGoal = Ji.shape[0]

        # get i-th task velocity
        dxGoali = dxGoalData[iTask].reshape(-1,1)

        # update variables for previous projection matrix and solution
        PiPrev = Pi
        dqiPrev = dqi

        # initialize variables for i-th task
        PiBar = PiPrev
        si = 1.0
        siStar = 0.0

        limitExceeded = True
        cntLoop = 1

        SactStar = Sact
        wStar = w

        PiBarStar = np.zeros_like(PiBar)
        PiHatStar = np.zeros((nJnt, nJnt + k))

        if np.sum(np.abs(dxGoali)) < tolTight:
            dqi = dqiPrev
            limitExceeded = False

        while limitExceeded:
            limitExceeded = False
            
            # update PiHat
            PiHat = (I - invSolver(Ji.dot(PiBar)).dot(Ji)).dot(pinv(Cact.dot(PiPrev), tol))            
            
            # compute a solution without task scale factor            
            dqi = dqiPrev + invSolver(Ji.dot(PiBar)).dot(dxGoali - Ji.dot(dqiPrev)) + \
                PiHat.dot((w - Cact.dot(dqiPrev)))

            # check whether the solution violates the limits
            z = C.dot(dqi)
            z = np.ravel(z)
            if np.any(z < (limLow - tol)) or np.any(z > (limUpp + tol)):
                limitExceeded = True

            # compute goal velocity projected
            xdGoalProj = C.dot(invSolver(Ji.dot(PiBar))).dot(dxGoali)

            if np.linalg.norm(xdGoalProj) < tol:
                # if the projected goal velocity is close to zero, set scale
                # factor to zero
                si = 0
            else:
                # compute the scale factor and identify the critical joint                
                a = xdGoalProj
                b = C.dot(dqi) - a.reshape(-1,1)
                
                marginL = limLow - np.ravel(b)
                marginU = limUpp - np.ravel(b)

                sMax = np.zeros(nJnt + k)
                sMin = np.zeros(nJnt + k)
                for iLim in range(nJnt + k):
                    if Sact[iLim, iLim] == 1:
                        sMax[iLim] = np.inf
                    else:
                        sMax[iLim], sMin[iLim] = \
                        findScaleFactor(marginL[iLim], marginU[iLim], a[iLim])

                # most critical limit index
                mclIdx = np.argmin(sMax)
                si = sMax[mclIdx]

            if np.isinf(si):
                si = 0

            # do the following only if the task is feasible and the scale
            # factor caculated is correct
            if (iTask == 1 or si > 0) and cntLoop < nIterationsMax:

                scaledDqi = dqiPrev + invSolver(Ji.dot(PiBar)).dot( \
                    (si * dxGoali - Ji.dot(dqiPrev))) + \
                    PiHat.dot(w - Cact.dot(dqiPrev))

                z = C.dot(scaledDqi)
                z = np.ravel(z)
                
                limitSatisfied = not (any(z < (limLow - tol)) or \
                        any(z > (limUpp + tol)))
                if si > siStar and limitSatisfied:
                    siStar = si
                    SactStar = Sact
                    wStar = w
                    PiBarStar = PiBar
                    PiHatStar = PiHat

                Sact[mclIdx, mclIdx] = 1
                Cact = Sact.dot(C)
                     
                w[mclIdx,0] = min(max(limLow[mclIdx], z[mclIdx]), limUpp[mclIdx])

                PiBar = PiPrev - pinv(Cact.dot(PiPrev), tol).dot(Cact.dot(PiPrev))

                taskRank = np.linalg.matrix_rank(Ji.dot(PiBar), tol)
                if taskRank < ndxGoal:
                    si = siStar
                    Sact = SactStar
                    Cact = Sact.dot(C)
                    w = wStar
                    PiBar = PiBarStar
                    PiHat = PiHatStar

                    dqi = dqiPrev + invSolver(Ji.dot(PiBar)).dot( \
                        (si * dxGoali - Ji.dot(dqiPrev))) + \
                        PiHat.dot(w - Cact.dot(dqiPrev))

                    limitExceeded = False
            else:  # if the current task is infeasible
                si = 0
                dqi = dqiPrev
                limitExceeded = False

                if cntLoop == nIterationsMax:
                    print('\n\tWarning: the maximum number of iteration has been reached!\n')
                    
            cntLoop = cntLoop + 1
            
            if (si > 0):
                # update nullspace projection 
                Pi = PiPrev - pinv(Ji.dot(PiPrev), tol).dot(Ji.dot(PiPrev))

        sData[iTask] = si
        
    dqi = np.ravel(dqi)
                    
    return dqi, sData, exitCode

In [111]:
# a test example 1
dqLow = np.array([-1.0103, -0.8920, -0.1339, -0.2942, -0.3012, -0.1649, -0.5868])
dqUpp = np.array([0.4284, 0.3409, 0.8095, 0.7777, 0.8616, 0.4395, 1.0497])

dxGoal = np.matrix([-2.4444])
J = np.matrix([-0.4222, -1.8365, 1.0291, -0.2697, -1.2750, 0.6093, -0.8351])

C = np.eye(7)

dxGoalData = [dxGoal]
JData = [J]

dq, sData, exitCode = snsIk_vel_mt_extended(C, dqLow, dqUpp, dxGoalData, JData)

print(dq.transpose())
print(sData)

[ 0.25916949  0.3409     -0.1339      0.16555664  0.78266485 -0.1649
  0.51263013]
[1.]


In [112]:
# a test example 2
dqLow = np.array([-0.2219, -0.8426, -0.2623, -0.9867, -0.8986, -0.7017, -0.8321])
dqUpp = np.array([0.4243, 0.5456, 0.2229, 0.1097, 0.6263, 0.4464, 1.0609])

dxGoal = np.matrix([-0.2684, 1.7619, 0.8914, 0.3309, 0.9741, 0.3486]).reshape(-1,1)
J = np.matrix([[-1.1006, 0.4406, -0.1178, -2.0108, 0.8874, -0.1945, -0.2487],
                   [0.3025, -0.6387, 0.3752, 1.4583, 0.6051, 1.2198, 0.7682],
                   [-0.3512, 0.7398, -0.2975, 0.4974, 0.9426, 0.3072, -0.0744],
                   [0.9677, 1.4896, 0.0837, 1.0651, -0.1419, 0.9940, -1.3034],
                   [-0.0786, 1.8665, -0.0553, -1.3375, 0.2977, -0.1946, 0.0459],
                   [0.3862, 1.2667, -0.5599, 1.6852, 0.4540, -0.1922, -0.7211]])

C = np.eye(7)

dxGoalData = [dxGoal]
JData = [J]

dq, sData, exitCode = snsIk_vel_mt_extended(C, dqLow, dqUpp, dxGoalData, JData)

print(dq.transpose())
print(sData)

[0.42310313 0.54471296 0.2229     0.1097     0.6249646  0.44551505
 1.0589153 ]
[0.9982134]


In [117]:
# a test example 3
C = np.array([[1.0000, 0, 0, 0, 0],
                   [0, 1.0000, 0, 0, 0],
                   [0, 0, 1.0000, 0, 0],
                   [0, 0, 0, 1.0000, 0],
                   [0, 0, 0, 0, 1.0000],
                   [0.3326, -0.4603, -0.3199, -1.4403, -0.1235],
                   [-1.6891, -0.1858, -0.2942, 1.2148, -0.7641],
                   [-1.6501, -1.4852, -0.0911, -0.2994, -0.0619]])

limLow = np.array([-0.5562, -0.1291, -0.2056, -0.8774, -0.3615, -1.1188, -1.1437, -2.3734])
limUpp = np.array([0.7918, 0.9109, 0.1555, 0.4370, 0.5385, 2.2716, 1.9543, 1.9874])

dxGoal1 = np.array([0.3077]).reshape(-1,1)
dxGoal2 = np.array([0.2220]).reshape(-1,1)
dxGoal3 = np.array([0.0592, 0.7906]).reshape(-1,1)
dxGoalData = [dxGoal1, dxGoal2, dxGoal3]

JData1 = np.array([[0.6189,   -0.7349,   -1.1174,    0.4183,   -0.0752]])
JData2 = np.array([[0.4169,    0.3999,   -0.7464,   -0.3311,   -0.9817]])
JData3 = np.array([[0.0320,    0.7764,   -1.2231,   -0.4920,   -0.4437],
                    [0.1972,    1.4892,  -0.6457,   -0.3752,    0.8036]])

JData = [JData1, JData2, JData3]

dq, sData, exitCode = snsIk_vel_mt_extended(C, limLow, limUpp, dxGoalData, JData)

print(dq.transpose())
print(sData)

[0.7258542  0.22856383 0.03342528 0.16917139 0.09274729]
[1.         1.         0.59826918]


In [127]:
# a test example 4
C = np.array([[0.0000, 0, 0, 0, 0, 0, 0],
                [0, 1.0000, 0, 0, 0, 0, 0],
                [0, 0, 1.0000, 0, 0, 0, 0],
                [0, 0, 0, 1.0000, 0, 0, 0],
                [0, 0, 0, 0, 1.0000, 0, 0],
                [0, 0, 0, 0, 0, 1.0000, 0],
                [0, 0, 0, 0, 0, 0, 1.0000],
                [2.5831, -1.4589, -2.0462, -0.5019, 0.0258, -1.1471, 0.0909],
                [0.4492, -0.0869, 0.6709, -0.3029, -0.9252, -0.2001, -0.5626]])

limLow = np.array([-1.0901, -0.2636, -0.6656, -0.1281, -0.2707, -0.1640, -0.7324, -1.9921, -1.8989])
limUpp = np.array([0.2368, 0.7959, 0.2604, 0.2006, 0.8430, 0.6154, 0.9774, 1.4382, 1.5937])

dxGoal1 = np.array([-0.6256, -4.2560, 1.4834, -7.1243, 1.0170]).reshape(-1,1)
dxGoal2 = np.array([0.9389, -1.0420, 1.2743]).reshape(-1,1)
dxGoal3 = np.array([-0.6824, -1.2050, 1.0490, -1.6531, -1.0726]).reshape(-1,1)
dxGoal4 = np.array([-3.3727, 2.6204, 0.1514, 0.4900, 0.9709]).reshape(-1,1)
dxGoalData = [dxGoal1, dxGoal2, dxGoal3, dxGoal4]

JData1 = np.array([[ 0.4541,  1.7162,  1.5037, -0.8138, -0.2825, -0.1714, -0.887 ],
                   [ 1.0754,  1.4371, -0.3326, -0.4781,  0.6634, -0.0003,  1.3016],
                   [-0.6276, -0.2844,  0.9   ,  1.5343, -0.3795,  1.1554, -0.7426],
                   [ 0.4554, -0.9754,  1.9622, -1.269 ,  0.9464,  1.4599, -0.1653],
                   [ 0.3039,  1.4197, -0.0893,  0.2334,  2.5169,  0.9857, -0.9497]])
JData2 = np.array([[ 0.4962,  0.1712, -1.0698, -1.3148,  0.8012, -0.0898,  0.3666],
       [ 0.3178, -0.6097,  0.144 , -2.0493, -0.37  , -0.1329,  0.1814],
       [ 0.3144, -0.9218, -2.7618,  0.7548,  1.0196, -1.3853,  0.9129]])
JData3 = np.array([[1.6132, -0.1794, 0.8694, 1.0935, 0.7685, 0.7062, 0.1641],
                   [0.9024, -1.3709, 0.8257, -0.6090, -0.6722, -0.6487, 0.1502],
                   [-0.1548, -0.4471, -0.8096, 1.4649, 0.4636, -0.0904, 0.1992],
                   [1.8043, -0.0105, 0.3497, -0.5596, -0.0299, 0.0961, 0.6819],
                   [-0.1509, -0.6952, -0.2443, -1.6654, 0.0039, -0.4221, -0.9587]])
JData4 = np.array([[-0.0838, -1.2514,  0.8942,  0.9348, -1.29  , -1.5779,  0.3657],
       [-0.471 , -0.0233, -0.3954,  0.4428,  0.3944,  1.7483,  0.0341],
       [-0.1522,  0.5406, -0.2119, -0.0378,  0.4032, -0.504 ,  0.4701],
       [ 0.5107, -0.3325,  0.7064, -0.8294,  0.7334, -0.6481,  0.8696],
       [ 0.9829,  0.5499,  0.8694,  0.0541,  1.5186, -1.1224,  1.8625]])

JData = [JData1, JData2, JData3, JData4]

dq, sData, exitCode = snsIk_vel_mt_extended(C, limLow, limUpp, dxGoalData, JData)

print(dq.transpose())
print(sData)

[-1.04958467  0.35354827 -0.6656      0.09661051 -0.17366559 -0.164
 -0.7324    ]
[0.35571016 0.         0.         0.        ]


In [127]:
# a test example 5
C = np.array([[1.0000, 0, 0, 0, 0, 0],
                   [0, 1.0000, 0, 0, 0, 0],
                   [0, 0, 1.0000, 0, 0, 0],
                   [0, 0, 0, 1.0000, 0, 0],
                   [0, 0, 0, 0, 1.0000, 0],
                   [0, 0, 0, 0, 0, 1.0000],
                   [0.5449, 0.0211, 1.4731, -0.2120, 0.0731, 2.9528],
                   [-0.0512, -0.0948, -0.8337, -0.6548, -1.5338, -0.6792],
                   [-0.9874, -0.7120, -0.7448, 0.3893, 0.4339, 0.2941]])

limLow = np.array([-1.0901, -0.2636, -0.6656, -0.1281, -0.2707, -0.1640, -0.7324, -1.9921, -1.8989])
limUpp = np.array([0.2368, 0.7959, 0.2604, 0.2006, 0.8430, 0.6154, 0.9774, 1.4382, 1.5937])

dxGoal1 = np.array([-0.6256, -4.2560, 1.4834, -7.1243, 1.0170]).reshape(-1,1)
dxGoal2 = np.array([0.9389, -1.0420, 1.2743]).reshape(-1,1)
dxGoal3 = np.array([-0.6824, -1.2050, 1.0490, -1.6531, -1.0726]).reshape(-1,1)
dxGoal4 = np.array([-3.3727, 2.6204, 0.1514, 0.4900, 0.9709]).reshape(-1,1)
dxGoalData = [dxGoal1, dxGoal2, dxGoal3, dxGoal4]

JData1 = np.array([[ 0.4541,  1.7162,  1.5037, -0.8138, -0.2825, -0.1714, -0.887 ],
                   [ 1.0754,  1.4371, -0.3326, -0.4781,  0.6634, -0.0003,  1.3016],
                   [-0.6276, -0.2844,  0.9   ,  1.5343, -0.3795,  1.1554, -0.7426],
                   [ 0.4554, -0.9754,  1.9622, -1.269 ,  0.9464,  1.4599, -0.1653],
                   [ 0.3039,  1.4197, -0.0893,  0.2334,  2.5169,  0.9857, -0.9497]])
JData2 = np.array([[ 0.4962,  0.1712, -1.0698, -1.3148,  0.8012, -0.0898,  0.3666],
       [ 0.3178, -0.6097,  0.144 , -2.0493, -0.37  , -0.1329,  0.1814],
       [ 0.3144, -0.9218, -2.7618,  0.7548,  1.0196, -1.3853,  0.9129]])
JData3 = np.array([[1.6132, -0.1794, 0.8694, 1.0935, 0.7685, 0.7062, 0.1641],
                   [0.9024, -1.3709, 0.8257, -0.6090, -0.6722, -0.6487, 0.1502],
                   [-0.1548, -0.4471, -0.8096, 1.4649, 0.4636, -0.0904, 0.1992],
                   [1.8043, -0.0105, 0.3497, -0.5596, -0.0299, 0.0961, 0.6819],
                   [-0.1509, -0.6952, -0.2443, -1.6654, 0.0039, -0.4221, -0.9587]])
JData4 = np.array([[-0.0838, -1.2514,  0.8942,  0.9348, -1.29  , -1.5779,  0.3657],
       [-0.471 , -0.0233, -0.3954,  0.4428,  0.3944,  1.7483,  0.0341],
       [-0.1522,  0.5406, -0.2119, -0.0378,  0.4032, -0.504 ,  0.4701],
       [ 0.5107, -0.3325,  0.7064, -0.8294,  0.7334, -0.6481,  0.8696],
       [ 0.9829,  0.5499,  0.8694,  0.0541,  1.5186, -1.1224,  1.8625]])

JData = [JData1, JData2, JData3, JData4]

dq, sData, exitCode = snsIk_vel_mt_extended(C, limLow, limUpp, dxGoalData, JData)

print(dq.transpose())
print(sData)

[-1.04958467  0.35354827 -0.6656      0.09661051 -0.17366559 -0.164
 -0.7324    ]
[0.35571016 0.         0.         0.        ]


In [129]:
# a test example 6
C = np.array([[1.0000, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 1.0000, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 1.0000, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1.0000, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1.0000, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 1.0000, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 1.0000, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1.0000, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1.0000],
              [-0.9687, -0.5112, -1.0504, -0.7716, 0.8954, 0.5086, -0.6689, -1.4198, 0.2010]])

limLow = np.array([-0.1975, -0.9917, -0.4040, -0.6569, -0.1661, -0.4440, -0.4345, -0.5518, -0.3733, -2.0548])
limUpp = np.array([0.8161, 0.4068, 0.5975, 0.5111, 0.5197, 0.2307, 0.7226, 0.7676, 1.0019, 1.5616])

dxGoal1 = np.array([3.4962, 1.3826, -2.1667, -0.7839, 1.5863]).reshape(-1,1)
dxGoal2 = np.array([-1.2212, 1.8002, 0.7748, 0.6240, 0.1569, -0.1440, 0.9097]).reshape(-1,1)
dxGoal3 = np.array([-7.8142, -7.0465, 7.1110, 2.1841, -4.6449, 4.0502]).reshape(-1,1)
dxGoalData = [dxGoal1, dxGoal2, dxGoal3]

JData1 = np.array([[1.7647, -1.1464, -0.0349, 0.8360, 0.0897, 0.9808, 1.0866, -1.9363, 1.4952],
    [-0.3469, 0.0918, -1.6526, 0.9208, 1.1554, 0.7923, -0.0407, 1.9477, 0.0889],
    [-1.0242, -0.3241, 0.6223, -0.1786, -0.2437, 0.3735, -0.0784, -0.9456, 0.5710],
    [0.7162, 0.0954, 0.0697, -1.0228, -0.9710, -0.6648, 0.6537, -0.4747, -1.6856],
    [2.0850, 0.1077, 1.5890, 0.0319, 0.0433, 0.6057, 1.2014, 0.8470, -0.3974]])

JData2 = np.array([[ 0.2629,  1.3305,  0.3111,  1.2653, -0.7244, -0.8525,  1.333 ,  1.1284, -0.7414],
                [-0.4705, -0.5695, -0.5214, -2.5347, -0.9381, -0.877 , -1.8179,  0.6409, -0.0818],
                [ 1.3701, -1.0038, -0.1122, -0.0239, -2.464 , -1.0634,  0.0789,  0.8488,  1.591 ],
                [ 0.6726, -0.7023,  0.5366, -0.2373,  1.138 ,  1.8606,  0.9562,  0.1482,  0.1223],
                [ 0.8711,  0.2292, -0.3414, -2.2698,  0.3104,  0.0078, -0.3557, -1.2746, -0.8684],
                [ 0.4121, -1.2288,  1.3605,  0.7509,  0.5309,  0.6629, -1.646 , -0.4388,  0.309 ],
                [-0.8504,  0.984 ,  0.7135, -0.586 , -2.3668,  0.0664, -1.2649,  0.8691,  0.5059]])
JData3 = np.array([[-1.3875, -0.3672,  1.4338,  1.7378, -0.9138, -0.4924, -1.6235, -0.2396, -0.9858],
       [ 0.0524,  1.6032, -0.2928,  0.9398, -1.1158, -0.0504, -1.714 , -1.0885, -0.3079],
       [ 2.3241, -1.0956, -1.9865,  0.8417,  1.6061,  1.027 ,  0.6433, -2.05  ,  1.6206],
       [ 1.2766, -1.2169,  1.0664,  0.0753,  0.2545,  0.2775, -1.5166, -0.2936,  1.3101],
       [-0.1737,  1.3947, -1.1504,  0.654 ,  0.0398,  0.7925, -0.661 , -2.3988,  0.22  ],
       [0.5503, 0.7196, -0.5224, -1.6097, -0.6524, -0.2681, 0.6123, -0.0186, 0.3332]])

JData = [JData1, JData2, JData3]

dq, sData, exitCode = snsIk_vel_mt_extended(C, limLow, limUpp, dxGoalData, JData)

print(dq.transpose())
print(sData)

[ 0.8161      0.4068     -0.404       0.5111      0.35041571 -0.444
  0.24628362 -0.08295814  0.40760797]
[0.5860224 0.        0.       ]


### Unit Test

In [153]:
import time 

def runTest_snsIk_vel_mt_extended(solver, nTest, optTol, cstTol, fid):
    # run the tests
    nPass = 0
    nFail = 0
    nSubOptimal = 0
    solveTime = np.zeros(nTest)
    sDataTotal = [None]*nTest
    
    np.random.seed(3247824738)
    
    for iTest in range(nTest):
        # print(f'--------------- TEST {iTest + 1} ---------------\n')
        
        # set up the problem dimensions
        k = np.random.randint(1, 3)
        nJnt = np.random.randint(k+2, 10)
        
        # set up the problem itself
        JLim = np.random.randn(k, nJnt) # jacobian for Cartesian limits
        
        dqLow = -0.1 - np.random.rand(nJnt)
        dqUpp = 0.1 + np.random.rand(nJnt)
        
        dxLow = -0.5 - 2*np.random.rand(k)
        dxUpp = 0.5 + 2*np.random.rand(k)
        
        nTask = 2 + np.random.randint(1, 3) # number of tasks
        ndxGoal = [None]*nTask
        scale = np.ones((nTask, 1))
        JData = [None]*nTask
        dqTmp = np.zeros((nJnt,nTask))
        dxGoalData = [None]*nTask
        
        for iTask in range(nTask):
            ndxGoal[iTask] = np.random.randint(1, nJnt-k) # task space dimension
            
            # set up the problem itself
            JData[iTask] = np.random.randn(ndxGoal[iTask], nJnt) # jacobian
            
            # compute the unscaled solution
            dqTmp[:,iTask] = dqLow + (dqUpp - dqLow) * np.random.rand(nJnt)
            
            for iJnt in range(nJnt):
                # saturate some joints to either limit at some probability
                if np.random.rand() < 0.2:
                    if np.random.rand() < 0.5:
                        dqTmp[iJnt,iTask] = dqLow[iJnt]
                    else:
                        dqTmp[iJnt,iTask] = dqUpp[iJnt]
            
            if np.random.random() < 0.2 and iTask > 1:
                dxGoalData[iTask] = np.zeros((ndxGoal[iTask],1))
            else:
                if np.random.random() < 0.4:
                    scale[iTask] = 0.1 + 0.8 * np.random.random()
                dxGoali = JData[iTask].dot(dqTmp[:,iTask]) / scale[iTask]
                dxGoalData[iTask] = dxGoali.transpose()
                
                
        C = np.vstack((np.eye(nJnt), JLim))
        limLow = np.hstack((dqLow, dxLow))
        limUpp = np.hstack((dqUpp, dxUpp))
        
        # solve the problem
        testPass = True
        startTime = time.time()        
        
#         print('C')
#         print(C)
#         print('limLow')
#         print(limLow)
#         print('limUpp')
#         print(limUpp)
#         print('dxGoalData')
#         print(dxGoalData)
#         print('JData')
#         print(JData)
        
        dq, sData, exitCode = solver(C, limLow, limUpp, dxGoalData, JData)
        solveTime[iTest] = time.time() - startTime
        sDataTotal[iTest] = sData
        sPrimaryTask = sDataTotal[iTest][0]
        # scalePrimaryTask = scale[0]
        
        # check the solution
        if exitCode != 1:
            print(f'\n  solver failed with exit code {exitCode}\n')
            testPass = False
        # if sPrimaryTask < scalePrimaryTask - optTol:
        #     print(f'\n  task scale is sub-optimal!  s ({sPrimaryTask:.6f}) < scale ({scalePrimaryTask:.6f})\n')
        #     nSubOptimal += 1
        z = C.dot(dq)
        if np.any(z > limUpp + cstTol) or np.any(z < limLow - cstTol):
            print('\n  constraints are violated!  [limLow, C*dq, limUpp]')
            testPass = False
        taskError = sPrimaryTask * dxGoalData[0] - JData[0].dot(dq)
        if np.any(np.abs(taskError) > cstTol):
            print('\n  task error (%.6f) exceeds tolerance!' % np.max(np.abs(taskError)))
            testPass = False
            
        if testPass:
            nPass = nPass + 1
            # print('  --> pass')
        else:
            nFail = nFail + 1
            print(f'--------------- TEST {iTest + 1} ---------------')
            print('  --> fail\n')

    print('\n')
    print('------------------------------------')
    print('nPass: %d  --  nFail: %d' % (nPass, nFail))
    print('------------------------------------')
    
    result = dict()
    result['nPass'] = nPass;
    result['nFail'] = nFail;
    result['nSubOptimal'] = nSubOptimal
    result['solveTime'] = solveTime
    result['sDataTotal'] = sDataTotal
    
    return result

In [154]:
# shared test parameters
optTol = 1e-4
cstTol = 1e-6
fid = 1
nTest = 1000

# extended SNS-IK solver algorithm for multiple tasks
result = runTest_snsIk_vel_mt_extended(snsIk_vel_mt_extended, nTest, optTol, cstTol, fid)
sDataTotal = result['sDataTotal']
sDataPrimary = np.zeros(nTest)
sDataSecondary = np.zeros(nTest)
numTaskData = np.zeros(nTest)
numTaskFeasibleData = np.zeros(nTest)
for i in range(nTest):
    sDataTmp = sDataTotal[i]
    sDataPrimary[i] = sDataTmp[0]
    sDataSecondary[i] = sDataTmp[1]
    numTaskData[i] = len(sDataTmp)
    numTaskFeasibleData[i] = len(np.where(np.array(sDataTmp) > 0)[0])
print(f"average primary task scale factor: {np.mean(sDataPrimary):.4f}")
print(f"average secondary task scale factor: {np.mean(sDataSecondary):.4f}")
print(f"average number of feasible tasks: ({np.mean(numTaskFeasibleData):.4f}/{np.mean(numTaskData):.4f})")
print(f"percentage of sub-optimal task scale factors: {result['nSubOptimal']/nTest*100:.2f}%")



------------------------------------
nPass: 1000  --  nFail: 0
------------------------------------
average primary task scale factor: 0.9004
average secondary task scale factor: 0.2775
average number of feasible tasks: (1.8600/3.5060)
percentage of sub-optimal task scale factors: 0.00%
